In [92]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import networkx as nx 
from node2vec import Node2Vec
from matplotlib import pyplot as plt, rc, cm
from matplotlib.pyplot import figure
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.manifold import TSNE
 from mpl_toolkits.mplot3d import Axes3D

In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data


In [ ]:
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Training

#loading data
isolated_nodes = pd.read_csv("isolated_nodes.csv")
node_classification = pd.read_csv("node_classification.csv")      #there are 4 unique page types
training_graph = pd.read_csv("training_graph.csv")
with open('node_features_text.json') as f:
   node_features_text = json.load(f)

In [ ]:
print(training_graph)

In [ ]:
print(node_classification)

In [ ]:
#Getting IDs for nodes that are of type 4
NC = node_classification.to_numpy()
print(np.shape(NC))
TG = training_graph.to_numpy()
G4 =[]
clr = []
k = 0
for i in range(np.shape(NC)[0]):
    if k<500:
      G4.append(NC[i][0])
      clr.append(NC[i][1])
      k = k+1;
    else:
      break;
print(len(G4))
print(len(clr))
clrs = []
for i in clr:
  if i == 1:
    clrs.append('red')
  if i == 2:
    clrs.append('limegreen')
  if i == 3:
    clrs.append('blue')
  if i == 4:
    clrs.append('magenta')
print(len(clrs))

In [ ]:
#Checking which nodes chosen are interconnected
G = nx.Graph()
print(np.shape(TG)[0])
percent = 0
for i in range(np.shape(TG)[0]):
  percent = percent + 1
  print(percent)
  if TG[i][0] in G4:
    if TG[i][1] in G4:
      G.add_edge(TG[i][0], TG[i][1])
nx.draw(G)
plt.show()

In [ ]:
!pip install -U node2vec

In [ ]:
G.name = "4th type"

#the vector size 
vector_size = round(len(G.nodes)**0.25)
setup = Node2Vec(G,dimensions=vector_size, walk_length=5, num_walks=5)
model = setup.fit(window=10, min_count=1)


In [ ]:
print(model)

In [ ]:
#Prepare the data to be processed 

G5 = G4
df1 = pd.DataFrame()
df1['Vertex']  = G4
df1['Label']  = G5
label = df1
G6 = []
df2 = df1 
for i in range(len(G5)):
  G6.append(1)
df2['Weight'] = G6
print(df2)

In [162]:
def df_to_graph(edges,name):
  graph=nx.Graph()
  graph.add_weighted_edges_from([tuple(x) for x in edges.values])
  graph.name = "The visualiazation of the subgraph" + " " + name
  print(nx.info(graph))
  print("------------------------------------")
  print("************************************")
  print("------------------------------------")
  return graph

In [ ]:
graph = df_to_graph(df1, "with 500 nodes")
print(df1)

In [ ]:
#Draw the graph
nx.draw_networkx(graph)

In [ ]:
#Prepare the data for visualization, all nodes has weight 1 since it's unweighted graph basically.

vector_size = round(len(graph.nodes)**0.25)
vector_size
label.drop('Weight',axis=1)
print(df2)

In [ ]:
#Do node vec for the nodes 
setup = Node2Vec(graph,dimensions=vector_size, walk_length=5, num_walks=5)
model = setup.fit(window=10, min_count=1)
print(model)

In [172]:
def Visualization(model):
    lbs = []
    tkns = []

    for wrd in model.wv.vocab:
        tkns.append(model[wrd])
        lbs.append(word)

    # print(len(labels))
    # print(labels)
    lab = []
    for i in range(len(labels)):
      lab.append(G4[i])
    
    tsne_model = TSNE(perplexity=10, n_components=3, learning_rate=20, init='random', n_iter=1000, random_state=42)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    z = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        z.append(value[2])
    
    #Setting the visualization environment 
    sns.set_style("whitegrid", {'axes.grid' : False})

    #Initializating the figure 
    fig = plt.figure(figsize=(20, 15))

    ax = Axes3D(fig)

    #Scatter all the nodes in a manner that closely connected nodes are close

    ax.scatter(x, y, z, c=clrs, marker='o',alpha=0.7,s=500)
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')
    ax.set_zlabel('Z coordinate')
    ax.set_title('Visualization of the subgraph of the bigger graphs, connected nodes are close in 3D. Different colors --> Different Page Type')
    ax.legend()

    for x_label, y_label, z_label, label_node in zip(x, y, z, lab):
      ax.text(x_label, y_label, z_label, label_node)

    ## 360 degree view
    def rotate(angle):
      ax.view_init(azim=angle)

    
    rc('animation', html='jshtml')
    vis = animation.FuncAnimation(fig, rotate, frames=np.arange(0,362,2),interval=100)
    vis.save('networkColor.gif', dpi=40, writer='pillow')

In [ ]:
Visualization(model)

In [ ]:
pip install pyvis

In [45]:
from pyvis.network import Network

In [ ]:
pip install vtk